In [21]:
import regex as re
import pandas as pd

In [22]:
f=open('./HDFS_2k.log')
lines=f.readlines()

In [23]:
split_pt=re.compile(r'([\s$@<>\[\]\'"=]|\((?!\))|(?<!\()\)|[:,\.](?=\s)|(?<=\b[a-zA-Z]+):|:(?=[a-zA-Z]+\b))')
hm_pt=re.compile(r'(.*?:)(.*)')

In [24]:
header=[]
msg=[]
for line in lines:
    h,m=hm_pt.match(line).groups()
    
    header.append(h)
    msg.append(m)

In [25]:
print(header[0])
print(msg[0])

081109 203615 148 INFO dfs.DataNode$PacketResponder:
 PacketResponder 1 for block blk_38865049064139660 terminating


In [26]:
header_tokens=[]
for h in header:
    tokens=[token for token in split_pt.split(h) if token!='' and token!=None]
    header_tokens.append(tokens)

In [27]:
msg_tokens=[]
for m in msg:
    tokens=[token for token in split_pt.split(m) if token!='' and token!=None]
    msg_tokens.append(tokens)

In [28]:
pd.DataFrame({'header_tokens':header_tokens,'msg_tokens':msg_tokens}).to_csv('tokens.csv')

In [39]:
header_anno=[]
for tokens in header_tokens:
    anno=['O']*len(tokens)
    anno[6],anno[8]='B-LVL','B-CLS'

    header_anno.append(anno)

In [40]:
list(zip(header_tokens[0],header_anno[0]))

[('081109', 'O'),
 (' ', 'O'),
 ('203615', 'O'),
 (' ', 'O'),
 ('148', 'O'),
 (' ', 'O'),
 ('INFO', 'B-LVL'),
 (' ', 'O'),
 ('dfs.DataNode', 'B-CLS'),
 ('$', 'O'),
 ('PacketResponder', 'O'),
 (':', 'O')]

In [41]:
msg_anno=[]
for tokens in msg_tokens:
    anno=['<O>']*len(tokens)
    msg_anno.append(anno)

In [49]:
re_date=r'\d{2}\/\d{2}\/\d{4}'
re_time=r'^\d{2}:\d{2}$'

re_obj=r'[a-zA-Z]+-\d{1,3}$'
re_cls=r'(\.([A-Z]+[a-z]*)+)|([A-Z]+[a-z]+){2,}'
re_func=r'\S*\(\)$'
re_path=r'^\'?\/[a-zA-Z]'
re_ip=r'(?:[0-9]{1,3}\.){3}[0-9]{1,3}'

re_host=r':\d{3,5}$'
re_url=r'[a-z]+:\/\/\S*'


In [50]:
def sub_tag(ntokens,ntags, patt_tag_list):

    def _sub_tag(ntokens,ntags, patt, target_tag):
        for tokens,tags in zip(ntokens,ntags):
            for i in range(len(tokens)):
                if re.search(patt,tokens[i]):
                    tags[i]=target_tag

    for patt,tag in patt_tag_list:
        _sub_tag(ntokens,ntags,patt,tag)

In [51]:
sub_tag(msg_tokens,msg_anno,
    [(re_cls,'B-CLS'),
    (re_func,'B-FUNC'),
    (re_path,'B-PATH'),
    (re_ip,'B-IP'),
    (re_host,'B-HOST'),
    (re_url,'B-URL')])

In [52]:
list(zip(msg_tokens[0],msg_anno[0]))

[(' ', '<O>'),
 ('PacketResponder', 'B-CLS'),
 (' ', '<O>'),
 ('1', '<O>'),
 (' ', '<O>'),
 ('for', '<O>'),
 (' ', '<O>'),
 ('block', '<O>'),
 (' ', '<O>'),
 ('blk_38865049064139660', '<O>'),
 (' ', '<O>'),
 ('terminating', '<O>')]

In [53]:
with open('./header_check.txt','w') as f:
    for tokens,tags in zip(header_tokens,header_anno):
        for token,tag in zip(tokens,tags):
            f.write(token+' '+tag+'\n')
        f.write('\n')

In [54]:
with open('./msg_check.txt','w') as f:
    for tokens,tags in zip(msg_tokens,msg_anno):
        for token,tag in zip(tokens,tags):
            f.write(token+' '+tag+'\n')
        f.write('\n')

In [55]:
df=pd.DataFrame({
    'tokens':[h+m for h,m in zip(header_tokens,msg_tokens)],
    'tags':[h+m for h,m in zip(header_anno,msg_anno)],
    'raw_log':[h+m for h,m in zip(header,msg)]
    })
df.to_csv('./hadoop_ds.csv')